In [75]:
import pandas as pd
import numpy as np
import re
from modlamp.descriptors import GlobalDescriptor
pd.options.mode.chained_assignment = None
Root = "C:/Users/JJvRijn/Documents/Python_Scripts"

In [2]:
#this file is freely availible from http://dramp.cpu-bioinfor.org/downloads/
DRAMP_data = pd.read_excel(Root + "/Programing/Data/AMP_focussed_data/general_amps.xls")

In [32]:
seq = DRAMP_data.Sequence
toxic = DRAMP_data.Hemolytic_activity
df_DRAMP = pd.DataFrame([seq, toxic], index = ['seq', 'toxic'])
df_DRAMP = df_DRAMP.transpose()
df_DRAMP.seq = list(map(lambda x: x.upper(),df_DRAMP.seq))
df_DRAMP

,seq,toxic
0,SLGPAIKATRQVCPKATRFVTVSCKKSDCQ,Not found
1,VTSWSLCTPGCTSPGGGSNCSFCC,Not found
2,GNGVLKTISHECNMNTWQFLFTCC,Not found
3,SSSGWLCTLTIECGTIICACR,Not found
4,MSWLNFLKYIAKYGKKAVSAAWKYKGKVLEWLNVGPTLEWVWQKLK...,Not found
...,...,...
5814,SDDAQRCYPHNRTPFTYTYI,[Ref.29307492] The peptide showed 3.3% hemolys...
5815,EPCSPKNNYHDLFYRT,[Ref.29307492] The peptide showed 1.7% hemolys...
5816,CNPLNGADRRTDSFPRFTVI,[Ref.29307492] The peptide showed 1.7% hemolys...
5817,TCRTNRPCFYDLDLNVCRCS,[Ref.29307492] No hemolytic activity informati...


In [380]:
NAA = ["A", "R", "N", "D", "C", "Q", "E", "G", "H", "I", "L", "K", "M", "F", "P", "S", "U",
       "T", "W", "Y", "V", " "]
unvalid =[]
for j in df_DRAMP.index:
    if (not all ([NAA_seq in NAA for NAA_seq in df_DRAMP.seq[j]])):
        unvalid.append(j)
df_DRAMP = df_DRAMP.drop(unvalid)
df_DRAMP = df_DRAMP.reset_index(drop=True)

In [33]:
df_DRAMP = df_DRAMP[df_DRAMP["toxic"] != "Not found"]
df_DRAMP = df_DRAMP[df_DRAMP["toxic"] != " No hemolytic activity information found."]
df_DRAMP = df_DRAMP[df_DRAMP["toxic"] != ' Not found in the literature']
df_DRAMP = df_DRAMP.dropna(subset=['toxic'])
for n in df_DRAMP.index:
    df_DRAMP.toxic[n] = re.sub('\[.*?\]', '', df_DRAMP.toxic[n])
df_DRAMP = df_DRAMP.reset_index(drop=True)
df_DRAMP

,seq,toxic
0,ATRSYGNGVYCNNSKCWVNWGEAKENIAGIVISGWASGLAGMGH,Non-hemolytic against calf blood.
1,NRWYCNSAAGGVGGAAVCGLAGYVGEAKENIAGEVRKGWGMAGGFT...,Non-hemolysis
2,GIPCGESCVFIPCITAAIGCSCKSKVCYRN,HD50=7.1 µM against human type A erythrocytes
3,GIPCGESCVFIPCITGAIGCSCKSKVCYRN,HD50=8.4 µM against human type A erythrocytes
4,GLPVCGETCVGGTCNTPGCTCSWPVCTRN,It has 20% hemolytic activity at 0.6 μM and 7...
...,...,...
1508,SDDAQRCYPHNRTPFTYTYI,The peptide showed 3.3% hemolysis against hum...
1509,EPCSPKNNYHDLFYRT,The peptide showed 1.7% hemolysis against hum...
1510,CNPLNGADRRTDSFPRFTVI,The peptide showed 1.7% hemolysis against hum...
1511,TCRTNRPCFYDLDLNVCRCS,No hemolytic activity information found.


In [142]:
df_DRAMP['toxic2'] = 'NaN'
df_DRAMP['toxic3'] = 'NaN'
df_DRAMP['toxic4'] = 'NaN'
new_values = pd.DataFrame(columns = df_DRAMP.columns)
for n in range(len(df_DRAMP)):
    if "-" in df_DRAMP.toxic[n]:
        df_DRAMP.loc[n, 'toxic']= re.sub('-[0-9. ]+[%%μµnm ]', '', df_DRAMP.toxic[n])
    if "±" in df_DRAMP.toxic[n]:
        df_DRAMP.loc[n, 'toxic']= re.sub('±[0-9. ]+[%%μµnm ]', '', df_DRAMP.toxic[n])    
    if "no hemolytic activity" in df_DRAMP.toxic[n]:
        df_DRAMP.loc[n, 'toxic'] = df_DRAMP.toxic[n].replace('no hemolytic activity', '0% hem')
    if "no detectable hemo" in df_DRAMP.toxic[n]:
        df_DRAMP.loc[n, 'toxic'] = df_DRAMP.toxic[n].replace('no detectable '
                                                             , '0%')
    if "MHC" in df_DRAMP.toxic[n]:
        if "MHC10" not in df_DRAMP.toxic[n]:
            df_DRAMP.loc[n, 'toxic'] = df_DRAMP.toxic[n].replace('MHC', '15% hem')
    if "Non-hemoly" in df_DRAMP.toxic[n]:
        df_DRAMP.loc[n, 'toxic'] = df_DRAMP.toxic[n].replace('Non-', '0%')        
    df_DRAMP.toxic2[n] = re.findall(r"[-+]?\d*\.\d+|\d+", str(df_DRAMP.toxic[n]))
    if (len(df_DRAMP.toxic2[n])==2):
        correct_row = df_DRAMP.loc[n]
        correct_row['toxic3'] = (df_DRAMP.toxic2[n])[0]
        correct_row['toxic4'] = (df_DRAMP.toxic2[n])[1]
        new_values.loc[len(new_values)] = correct_row            
    if (len(df_DRAMP.toxic2[n])>2):
        if (len(df_DRAMP.toxic2[n])%2 == 0):
            single_pep = df_DRAMP.toxic[n].replace('and',',').replace('##',',').replace(';',',').split(sep =",")
            for r in single_pep:
                two_things = re.findall(r"[-+]?\d*\.\d+|\d+", str(r))                
                if len(two_things) == 2:
                    if "positive control" not in r:
                        correct_row = df_DRAMP.loc[n]
                        correct_row['toxic'] = r
                        correct_row['toxic2'] = two_things
                        correct_row['toxic3'] = two_things[0]
                        correct_row['toxic4'] = two_things[1]
                        new_values.loc[len(new_values)] = correct_row
                if len(two_things) == 4:
                    if two_things[2] == "10":
                        correct_row = df_DRAMP.loc[n]
                        correct_row['toxic'] = r
                        correct_row['toxic2'] = two_things
                        correct_row['toxic3'] = (df_DRAMP.toxic2[n])[0]
                        correct_row['toxic4'] = float((df_DRAMP.toxic2[n])[1])*10**float((df_DRAMP.toxic2[n])[3])
                        new_values.loc[len(new_values)] = correct_row
df_DRAMP = new_values

,seq,toxic,toxic2,toxic3,toxic4
0,GIPCGESCVFIPCITAAIGCSCKSKVCYRN,HD50=7.1 µM against human type A erythrocytes,"[50, 7.1]",50,7.1
1,GIPCGESCVFIPCITGAIGCSCKSKVCYRN,HD50=8.4 µM against human type A erythrocytes,"[50, 8.4]",50,8.4
2,GLPVCGETCVGGTCNTPGCTCSWPVCTRN,It has 20% hemolytic activity at 0.6 μM,"[20, 0.6]",20,0.6
3,GLPVCGETCVGGTCNTPGCTCSWPVCTRN,75% hemolytic activity at 1.5 μM against huma...,"[75, 1.5]",75,1.5
4,GLPVCGETCVGGTCNTPGCTCSWPVCTRN,HD50=20.0M against Human red blood cells .,"[50, 20.0]",50,20.0
...,...,...,...,...,...
10006,SDDAQRCYPHNRTPFTYTYI,The peptide showed 3.3% hemolysis against hum...,"[3.3, 50]",3.3,50
10007,EPCSPKNNYHDLFYRT,The peptide showed 1.7% hemolysis against hum...,"[1.7, 50]",1.7,50
10008,EPCSPKNNYHDLFYRT,The peptide showed 1.7% hemolysis against hum...,"[1.7, 50]",1.7,50
10009,CNPLNGADRRTDSFPRFTVI,The peptide showed 1.7% hemolysis against hum...,"[1.7, 50]",1.7,50


In [371]:
df_DRAMP['toxic5'] = 'NaN'
for n in range(len(df_DRAMP)):
    if "M" in df_DRAMP.toxic[n]:
        df_DRAMP.loc[n, 'toxic5'] = "M"
    if "g/ml" in df_DRAMP.toxic[n]:
        df_DRAMP.loc[n, 'toxic5'] = "g/ml"
    if "g/L" in df_DRAMP.toxic[n]:
        df_DRAMP.loc[n, 'toxic5'] = "mg/ml" 
    if "mol/L" in df_DRAMP.toxic[n]:
        df_DRAMP.loc[n, 'toxic5'] = "M"   
    if "μM" in df_DRAMP.toxic[n]:
        df_DRAMP.loc[n, 'toxic5'] = "μM"
    if "µM" in df_DRAMP.toxic[n]:
        df_DRAMP.loc[n, 'toxic5'] = "μM"
    if "µmol/L" in df_DRAMP.toxic[n]:
        df_DRAMP.loc[n, 'toxic5'] = "μM"
    if "μmol/L" in df_DRAMP.toxic[n]:
        df_DRAMP.loc[n, 'toxic5'] = "μM"        
    if "µg/ml" in df_DRAMP.toxic[n]:
        df_DRAMP.loc[n, 'toxic5'] = "μg/ml"
    if "μg/ml" in df_DRAMP.toxic[n]:
        df_DRAMP.loc[n, 'toxic5'] = "μg/ml"
    if "μg/mL" in df_DRAMP.toxic[n]:
        df_DRAMP.loc[n, 'toxic5'] = "μg/ml"
    if "µg/mL" in df_DRAMP.toxic[n]:
        df_DRAMP.loc[n, 'toxic5'] = "μg/ml"        
    if "nM" in df_DRAMP.toxic[n]:
        df_DRAMP.loc[n, 'toxic5'] = "nM"       
    if "mg/L" in df_DRAMP.toxic[n]:
        df_DRAMP.loc[n, 'toxic5'] = "mg/L"
    if "mg/ml" in df_DRAMP.toxic[n]:
        df_DRAMP.loc[n, 'toxic5'] = "mg/ml"
    if "ng/ml" in df_DRAMP.toxic[n]:
        df_DRAMP.loc[n, 'toxic5'] = "ng/ml" 
    if "nmol/ml" in df_DRAMP.toxic[n]:
        df_DRAMP.loc[n, 'toxic5'] = "μM" 
    if "mM" in df_DRAMP.toxic[n]:
        df_DRAMP.loc[n, 'toxic5'] = "mM"              
df_DRAMP= df_DRAMP[df_DRAMP.toxic5 != "NaN"]
df_DRAMP = df_DRAMP.reset_index(drop=True)

In [385]:
glob = GlobalDescriptor((df_DRAMP.seq.tolist()))
glob.calculate_MW(amide=True)
df_DRAMP['MW'] = glob.descriptor
#see what concentrations are in µg/ml instead of µM
mask0 = df_DRAMP["toxic5"] == 'μg/ml'
mask1 = df_DRAMP["toxic5"] == 'mg/L'
mask2 = df_DRAMP["toxic5"] == 'mg/ml'
mask3 = df_DRAMP["toxic5"] == 'μM'
mask4 = df_DRAMP["toxic5"] == 'nM'
mask5 = df_DRAMP["toxic5"] == 'mM'
mask6 = df_DRAMP["toxic5"] == 'M'
mask7 = df_DRAMP["toxic5"] == 'g/ml'
mask8 = df_DRAMP["toxic5"] == 'ng/ml'
#calculate the µM
df_DRAMP["CONCENTRATION_µM"] = "NaN"
df_DRAMP.loc[mask0, "CONCENTRATION_µM"] = (
    (df_DRAMP["toxic4"][mask0]).astype(float)*1000) / df_DRAMP[mask0]['MW']
df_DRAMP.loc[mask1, "CONCENTRATION_µM"] = (
    (df_DRAMP["toxic4"][mask1]).astype(float)*1000) / df_DRAMP[mask1]['MW']
df_DRAMP.loc[mask2, "CONCENTRATION_µM"] = (
    (df_DRAMP["toxic4"][mask2]).astype(float)*1000000) / df_DRAMP[mask2]['MW']
df_DRAMP.loc[mask4, "CONCENTRATION_µM"] = (
    (df_DRAMP["toxic4"][mask4]).astype(float)/1000)
df_DRAMP.loc[mask5, "CONCENTRATION_µM"] = (
    (df_DRAMP["toxic4"][mask5]).astype(float)*1000)
df_DRAMP.loc[mask6, "CONCENTRATION_µM"] = (
    (df_DRAMP["toxic4"][mask6]).astype(float)*1000000)
df_DRAMP.loc[mask7, "CONCENTRATION_µM"] = (
    (df_DRAMP["toxic4"][mask7]).astype(float)*1000000000) / df_DRAMP[mask7]['MW']
df_DRAMP.loc[mask8, "CONCENTRATION_µM"] = (
    (df_DRAMP["toxic4"][mask8]).astype(float)) / df_DRAMP[mask8]['MW']
#fill in values already in µM
df_DRAMP.loc[mask3, "CONCENTRATION_µM"] = df_DRAMP["toxic4"][mask3].astype(float)

In [390]:
#HAPPENN formating of positive/negative
#happenn formating
df_DRAMP["toxic6"] = "~"
for v in range(len(df_DRAMP)):
    if float(df_DRAMP["toxic3"][v]) <= 5:
        if (df_DRAMP["CONCENTRATION_µM"][v]) >= 30:
            df_DRAMP.loc[v, "toxic6"] = 0        
    if 5 <= float(df_DRAMP["toxic3"][v]) <= 10:
        if (df_DRAMP["CONCENTRATION_µM"][v]) >= 60:
            df_DRAMP.loc[v, "toxic6"] = 0   
    if 10 <= float(df_DRAMP["toxic3"][v]) <= 15:
        if (df_DRAMP["CONCENTRATION_µM"][v]) >= 90:
            df_DRAMP.loc[v, "toxic6"] = 0
    if 15 <= float(df_DRAMP["toxic3"][v]) <= 20:
        if (df_DRAMP["CONCENTRATION_µM"][v]) >= 120:
            df_DRAMP.loc[v, "toxic6"] = 0
    if 20 <= float(df_DRAMP["toxic3"][v]) <= 25:
        if (df_DRAMP["CONCENTRATION_µM"][v]) >= 180:
            df_DRAMP.loc[v, "toxic6"] = 0  
    if 25 <= float(df_DRAMP["toxic3"][v]) <= 30:
        if (df_DRAMP["CONCENTRATION_µM"][v]) >= 180:
            df_DRAMP.loc[v, "toxic6"] = 0  
    if 30 <= float(df_DRAMP["toxic3"][v]) <= 35:
        if (df_DRAMP["CONCENTRATION_µM"][v]) >= 210:
            df_DRAMP.loc[v, "toxic6"] = 0         
    if 35 <= float(df_DRAMP["toxic3"][v]) <= 40:
        if (df_DRAMP["CONCENTRATION_µM"][v]) >= 240:
            df_DRAMP.loc[v, "toxic6"] = 0  
    if 40 <= float(df_DRAMP["toxic3"][v]) <= 45:
        if (df_DRAMP["CONCENTRATION_µM"][v]) >= 270:
            df_DRAMP.loc[v, "toxic6"] = 0  
    if float(df_DRAMP["toxic3"][v]) >= 50:
        if (df_DRAMP["CONCENTRATION_µM"][v]) <= 300:
            df_DRAMP.loc[v, "toxic6"] = 1
    if float(df_DRAMP["toxic3"][v]) >= 55:
        if (df_DRAMP["CONCENTRATION_µM"][v]) <= 330:
            df_DRAMP.loc[v, "toxic6"] = 1
    if float(df_DRAMP["toxic3"][v]) >= 60:
        if (df_DRAMP["CONCENTRATION_µM"][v]) <= 360:
            df_DRAMP.loc[v, "toxic6"] = 1
    if float(df_DRAMP["toxic3"][v]) >= 65:
        if (df_DRAMP["CONCENTRATION_µM"][v]) <= 390:
            df_DRAMP.loc[v, "toxic6"] = 1 
    if float(df_DRAMP["toxic3"][v]) >= 70:
        if (df_DRAMP["CONCENTRATION_µM"][v]) <= 420:
            df_DRAMP.loc[v, "toxic6"] = 1 
    if float(df_DRAMP["toxic3"][v]) >= 75:
        if (df_DRAMP["CONCENTRATION_µM"][v]) <= 450:
            df_DRAMP.loc[v, "toxic6"] = 1 
    if float(df_DRAMP["toxic3"][v]) >= 80:
        if (df_DRAMP["CONCENTRATION_µM"][v]) <= 480:
            df_DRAMP.loc[v, "toxic6"] = 1 
    if float(df_DRAMP["toxic3"][v]) >= 85:
        if (df_DRAMP["CONCENTRATION_µM"][v]) <= 510:
            df_DRAMP.loc[v, "toxic6"] = 1 
    if float(df_DRAMP["toxic3"][v]) >= 90:
        if (df_DRAMP["CONCENTRATION_µM"][v]) <= 540:
            df_DRAMP.loc[v, "toxic6"] = 1
    if float(df_DRAMP["toxic3"][v]) >= 95:
        if (df_DRAMP["CONCENTRATION_µM"][v]) <= 570:
            df_DRAMP.loc[v, "toxic6"] = 1
    if float(df_DRAMP["toxic3"][v]) >= 100:
        if (df_DRAMP["CONCENTRATION_µM"][v]) <= 600:
            df_DRAMP.loc[v, "toxic6"] = 1 
df_DRAMP = df_DRAMP[df_DRAMP["toxic6"]!="~"]
df_DRAMP = df_DRAMP.reset_index(drop=True)

In [397]:
peptides = df_DRAMP["seq"].unique()
final_data = pd.DataFrame(peptides, columns=['seq'])
for a in range(len(df_DRAMP)):
    for n in range(len(peptides)):
        toxic = []
        toxic_count = 0
        if df_DRAMP["seq"][a] == peptides[n]:
            toxic.append(float(df_DRAMP["toxic6"][a]))
            toxic_count = sum(toxic)/len(toxic)
            if toxic_count == len(toxic)/2:
                final_data.loc[n, "toxic"] = "~"
            if toxic_count <= len(toxic)/2:
                final_data.loc[n, "toxic"] = 0
            if toxic_count >= len(toxic)/2:
                final_data.loc[n, "toxic"] = 1

In [423]:
df_DRAMP['Length'] = df_DRAMP['seq'].str.len()
df_DRAMP = df_DRAMP.loc[(df_DRAMP.Length <= 35)]
df_DRAMP = df_DRAMP.loc[(df_DRAMP.Length >= 7)]

In [422]:
df_DRAMP.Length

0       30
1       30
2       19
3       23
4       16
        ..
1719    20
1720    16
1721    16
1722    20
1723    20
Name: Length, Length: 1724, dtype: int64

In [424]:
final_data.to_csv(
    Root + "/Programing/Data/AMP_focussed_data/DRAMP_data.csv",index = False, header=True)

In [427]:
All_data = pd.read_csv(Root + "/" + "Programing/Data/AMP_focussed_data/combined_final.csv")
data_complete = pd.concat([All_data, final_data])


In [436]:
new_peptides = list(set(data_complete.seq) - set(All_data.seq))
new_df = final_data[final_data["seq"].isin(new_peptides)]
new_df.to_csv(
    Root + "/Programing/Data/AMP_focussed_data/DRAMP_ind_data.csv",index = False, header=True)

In [438]:
new_df = new_df.reset_index(drop=True)
#Convert to FASTA
ofile = open(Root + "/" + "Programing/Data/DRAMP_fasta.txt", "w")

for i in range(len(new_df.seq)):
    ofile.write(">peptide" + str(i) + "\n" +new_df.seq[i] + "\n")

#do not forget to close it
ofile.close()

In [439]:
len(new_df)

371

In [449]:
df_DRAMP.loc[2,"toxic"]

'7% hemolytic activity at 250 μg/mL against human erythrocytes'